# Librerias a utilizar 

In [ ]:
!pip install -U spacy
!pip install networkx
!pip install fuzzywuzzy
#modelos NER (reconocimiento de entidades)
!python -m spacy download es_core_news_sm
!python -m spacy download es_core_news_md
#Streamlite para visualizar
#Local tunnel como puerto para visualizar el Streamlite, en modo local se puede ejecutar en localhost
!pip install streamlit==1.3.0
!npm install -g localtunnel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.9 MB 5.0 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 42.3 MB 1.4 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_md')
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 9.2 MB 5.4 MB/s 
     |███████████████████████

/tools/node/bin/lt -> /tools/node/lib/node_modules/localtunnel/bin/lt.js
+ localtunnel@2.0.2
added 22 packages from 22 contributors in 2.501s


## Parametros a modificar 

> ruta_archivo_interes = 

> numero_de_noticias_a_utilizar =

In [ ]:
#Escribimos un archivo de nombre 'app' con formato '.py' que pueda ser lanzado desde streamlite
%%writefile app.py 

#Parametros a modificar para que funcione en cualquier google collab o entorno adecuadamente configurado
#Carga de base de datos proporcionada por metrikabi
ruta_archivo_interes = '/content/drive/MyDrive/estancia_preprofesional/CIMAT_ESTANCIA.xlsx'

#Numero de noticias a utilizar del dataset o noticias a utilizar despues del filtro
numero_de_noticias_a_utilizar = 30
#########################################################
############################################################
# para preprocesamiento de lenguaje natural
import spacy
import en_core_web_sm  
# para visualizaciones
import streamlit as st
import matplotlib.pyplot as plt
import pandas as pd
import openpyxl
import numpy as np
import pickle
from operator import itemgetter
from fuzzywuzzy import process, fuzz
#%matplotlib inline
from matplotlib.pyplot import figure 

#diferentes opciones para visualizar grafos
import networkx as nx
import matplotlib.pyplot as plt
import graphviz as graphviz

###Paquetes de plotly
import pandas as pd
import plotly.offline as py
import plotly.graph_objects as go
import networkx as nx
import re


##Carga de modelo
##Podemos cargar el modelo small , el modelo median o un modelo entrenado por nosotros 
#import es_core_news_sm
#nlp = es_core_news_sm.load()
import es_core_news_md
nlp = es_core_news_md.load()

###
st.write('# ANALIS DE ENTIDADES ')
st.write('## PARA LA DETECCIÓN DE RIESGOS SOCIALES ')
###

try:
  df=pd.read_excel(ruta_archivo_interes)  #Shape(13896, 13)
  st.success('Archivo cargado correctamente')
except :
  e=RuntimeError('No se especifico la ruta del archivo o el archivo no se puede leer')
  st.exception(e)

df=df[df['Nota'].notnull()]   # 13,884 Noticias
df=df[df['Riesgo'].notnull()]           # 13,884 Noticias
tipo_de_riesgos=np.unique(df['Riesgo'])
riesgo_select=st.sidebar.selectbox('Selecciona el Riesgo',list(tipo_de_riesgos))
##Filtrando df por riesgo
df_filtro_riesgo = df[df['Riesgo']==riesgo_select]
##Filtrando df por Estado
estado_de_interes=list(['Todos'])
estado_de_interes=estado_de_interes+list(np.unique(df_filtro_riesgo['Estado']))
estado_select=st.sidebar.selectbox('Selecciona el Estado',estado_de_interes)
if estado_select !='Todos':
    df_filtro_estado = df_filtro_riesgo[df_filtro_riesgo['Estado']==estado_select]
    df_notas = df_filtro_estado['Nota']
else:
    df_notas = df_filtro_riesgo['Nota']
#st.write('El conjunto de datos es: ' ,df_notas.head())
#filtro para elegir,
entidades= st.multiselect('SELECCIONAR ENTIDADES DE INTERES  -PER:PERSONAS,-LOC:LOCACIONES,ORG:ORGANIZACIONES',
                          ('PER','LOC','ORG'))
#########################################
entities = []

for article in df_notas[0:30]:
    tokens = nlp(''.join(article))
    gpe_list = []
    for ent in tokens.ents:
      if ent.label_ in entidades:
          gpe_list.append(ent.text)
    entities.append(gpe_list)
#Limpiar tokens relacionados a entidades encontrados
articles = []
for entity_list in entities:
    cleaned_entity_list = []
    for entity in entity_list:
      a=(entity.lstrip('Los').replace("'s", "").replace("’s",""))
      b=a.lstrip('Respecto').lstrip("‘").lstrip("¡Basta").lstrip('“').lstrip('’')
      cleaned_entity_list.append(b)
    articles.append(cleaned_entity_list)


##Eliminar las noticias vacias
articles = [article for article in articles if article != []]

#Crear los nodos de un grafo
G = nx.Graph()
for entities in articles:
    if len(entities)>3:
      G.add_nodes_from(entities,size=1)

#Crear las relaciones entre los nodos
for entities in articles:
    if len(entities) >3:
        for i in range(len(entities)-1):
           G.add_edges_from([(str(entities[i]),str(entities[i+1]))],weight=1)

#########################################
degree_dict = dict(G.degree(G.nodes()))
nx.set_node_attributes(G, degree_dict, 'degree')
sorted_degree = sorted(degree_dict.items(),key=itemgetter(1), reverse=True)
#########################################
for i,j in sorted_degree[50:]:
  G.remove_node(i)

########################################Comienza Plotly
midsummer=G
pos_ = nx.spring_layout(midsummer)
def make_edge(x, y, text, width):    
    '''Creates a scatter trace for the edge between x's and y's with given width
    Parameters
    ----------
    x    : a tuple of the endpoints' x-coordinates in the form, tuple([x0, x1, None])
    y    : a tuple of the endpoints' y-coordinates in the form, tuple([y0, y1, None])
    width: the width of the line
    Returns
    -------
    An edge trace that goes between x0 and x1 with specified width.
    '''
    return  go.Scatter(x         = x,
                       y         = y,
                       line      = dict(width = width,
                                   color = '#888'),
                       hoverinfo = 'text',
                       text      = ([text]),
                       mode      = 'lines')
# For each edge, make an edge_trace, append to list
edge_trace = []
for edge in midsummer.edges():
    if midsummer.edges()[edge]['weight'] > 0:
        char_1 = edge[0]
        char_2 = edge[1]
        x0, y0 = pos_[char_1]
        x1, y1 = pos_[char_2]
        text   = char_1 + '--' + char_2 + ': ' + str(midsummer.edges()[edge]['weight'])
        trace  = make_edge([x0, x1, None], [y0, y1, None], text,
                           1*midsummer.edges()[edge]['weight']**1.75)
        edge_trace.append(trace)

# Make a node trace
node_trace = go.Scatter(x         = [],
                        y         = [],
                        text      = [],
                        textposition = "top center",
                        textfont_size = 10,
                        mode      = 'markers+text',
                        hoverinfo = 'text',
                        marker    = dict(colorscale='YlGnBu',
                                         color = [],
                                         size  = [],
                                         line  = None))
# For each node in midsummer, get the position and size and add to the node_trace
for node in midsummer.nodes():
    x, y = pos_[node]
    node_trace['x'] += tuple([x])
    node_trace['y'] += tuple([y])
    node_trace['marker']['color'] += tuple(['cornflowerblue'])
    node_trace['marker']['size'] += tuple([8*midsummer.nodes()[node]['size']])
    node_trace['text'] += tuple(['<b>' + node + '</b>'])
    
layout = go.Layout(
#Si se quiere modificar el color del fondo
#    paper_bgcolor='rgba(0,0,0,0)',
#    plot_bgcolor='rgba(0,0,0,0)',
margin=dict(b=20,l=20,r=20,t=30)
)
fig = go.Figure(layout = layout)
for trace in edge_trace:
    fig.add_trace(trace)
fig.add_trace(node_trace)
fig.update_layout(showlegend = False)
fig.update_xaxes(showticklabels = False)
fig.update_yaxes(showticklabels = False)
st.plotly_chart(fig,use_container_width=True)#, 
#fig.show()
try:
  col1, col2= st.columns(2)
  col1.metric("% de Noticias utilizadas", np.round((len(df_notas[:numero_de_noticias_a_utilizar])*100)/len(df_notas),2) )
  col2.metric("% de entidades Mostradas", np.round((len(sorted_degree[:50])*100)/len(sorted_degree),2))
  col3, col4= st.columns(2)
  col3.metric("Noticias utilizadas", len(df_notas[:numero_de_noticias_a_utilizar]) )
  col4.metric("Total de noticias", len(df_notas) )
except:
  e=RuntimeError('Aun no se seleccionada nada')
  st.exception(e)
#El porcentaje de sorted_degree 

Overwriting app.py


# Ejecución final

> Se genera un link 'your url is:', el cual puede ser compartido a cualquier equipo con acceso a internet para visualizar nuestra aplicación.

In [ ]:
#Activar el puerto 8501 simulando que estamos corriendo Streamlite de manera local.
!streamlit run app.py & npx lt --port 8501

your url is: https://giant-papayas-know-35-222-215-210.loca.lt
2022-05-30 03:30:27.961 INFO    numexpr.utils: NumExpr defaulting to 2 threads.

  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://35.222.215.210:8501

/usr/local/lib/python3.7/dist-packages/spacy/util.py:845: UserWarning: [W094] Model 'en_core_web_sm' (2.2.5) specifies an under-constrained spaCy version requirement: >=2.2.2. This can lead to compatibility problems with older versions, or as new spaCy versions are released, because the model may say it's compatible when it's not. Consider changing the "spacy_version" in your meta.json to a version range, with a lower and upper pin. For example: >=3.3.0,<3.4.0
  warnings.warn(warn_msg)
/usr/local/lib/python3.7/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Insta